In [ ]:
#default_exp torchcirq

In [ ]:
#export
import numpy as np
import os
from matplotlib import pyplot as plt

import torch
from torch.autograd import Variable
from complex import *
import qutip.qip.circuit as QCirc
import qutip as qt
from tqdm import tqdm as tqdm

In [ ]:
#hide
device = torch.device('cuda:0')
device = "cpu"

In [ ]:
#export
def generate_circle_data(samples, center=[0.0, 0.0], radius=np.sqrt(2 / np.pi)):
    """Generate cirle data"""
    Xvals, yvals = [], []

    for i in range(samples):
        x = 2 * (np.random.rand(2)) - 1
        y = 0
        if np.linalg.norm(x - center) < radius:
            y = 1
        Xvals.append(x)
        yvals.append(y)
    return np.array(Xvals), np.array(yvals)


def plot_data(x, y, fig=None, ax=None):
    """Plot Data with label colors"""
    if fig == None:
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    reds = y == 0
    blues = y == 1
    ax.scatter(x[reds, 0], x[reds, 1], s=15, edgecolor="k")
    ax.scatter(x[blues, 0], x[blues, 1], s=15, edgecolor="k")
    ax.set_xlabel("$x_1$")
    ax.set_ylabel("$x_2$")

In [ ]:
#hide
def iterate_minibatches(inputs, targets, batch_size):
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

X = np.array([[0,1],[1,0]])
Y = np.array([[0, -1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
unity = make_complex(np.eye(2))
cz = np.eye(4)
cz[3][3] = -1


def R1(angle, matrix):
    return torch.cos(angle/2).to(device)*unity.to(device) + torch.sin(angle/2).to(device)*make_complex(matrix*1j).to(device)

def R3(angles):
    return matmul(matmul(R1(angles[0], Z), R1(angles[1], Y)), R1(angles[2], Z))

def CZ(width, c=None, t=None):
    a = QCirc.csign(N=width, control=c, target=t).data
    return a.todense()

def Measurements(width, device="cpu"):
    ket1 = qt.basis(2, 0)
    ket2 = qt.basis(2, 1)
    a = qt.tensor(ket1*ket1.dag(), qt.qeye(2**(width-1))).data
    b = qt.tensor(ket2*ket2.dag(), qt.qeye(2**(width-1))).data
    return make_complex(a.todense()).to(device),make_complex(b.todense()).to(device) # Define measurement directions

def Measurement_Z(width, device="cpu"):
    Z = qt.sigmaz()
    a = qt.tensor(Z, qt.qeye(2**(width-1))).data
    return make_complex(a.todense()).to(device)

def cost(x):
    x_star = conj(x)
    return 1-real(inner_prod(x, x_star))**2

def flatten_grad(grad):
    tuple_to_list = []
    for tensor in grad:
        tuple_to_list.append(tensor.view(-1))
    all_flattened = torch.cat(tuple_to_list)
    return all_flattened

def find_hessian(loss, params, x=None):
    grad1 = torch.autograd.grad(loss, params, create_graph=True, allow_unused=True) #create graph important for the gradients

    grad1 = flatten_grad(grad1)
    list_length = grad1.size(0)
    hessian = torch.zeros(list_length, list_length)
    progress = tqdm(range(list_length))
    for idx in progress:
        print(idx, "/", list_length)
        grad2rd = torch.autograd.grad(grad1[idx], params, create_graph=True)
        cnt = 0
        hess_idx = 0
        for g in grad2rd:
            hess = g.contiguous().view(-1)
            g2 = hess if cnt == 0 else torch.cat([g2, hess])
            cnt = 1
        hessian[idx] = g2.detach().cpu()
        del g2

    H = hessian.cpu().data.numpy()
    return H

def find_heigenvalues(loss, params):
    H = find_hessian(loss, params)
    eigenvalues = np.linalg.eigvalsh(H)
    return eigenvalues, H

def loss_function(circ, params, x, init, target):
    out1 = matmul(circ(params, x=x), init)
    out1 = inner_prod(target, out1)
    out1_star = conj(out1)
    out2 = 1-torch.abs(real(inner_prod(out1, out1_star)))
    return out2

def batch_loss_function_sigmoid(circ, params, x_train, y_train, init, measurements=None):
    loss = 0.0
    for x,y in zip(x_train, y_train):
        idx = int((y + 1)/2)
        out1 = matmul(circ(params, x=x), init)
        out1_copy = out1.clone().detach().requires_grad_(True)
        out2 = matmul(measurements[idx], out1)
        out1 = inner_prod(out1_copy, out2) # this is already <Psi|0><0|Psi>
        #out1_star = conj(out1)
        out2 = torch.sigmoid(1-torch.abs(real(out1))).float()
        loss = loss + out2
    return loss/len(x_train)

def batch_loss_function(circ, params, x_train, y_train, init, measurements=None):
    loss = 0.0
    for x,y in zip(x_train, y_train):
        idx = int((y + 1)/2)
        out1 = matmul(circ(params, x=x), init)
        out1_copy = out1.clone().detach().requires_grad_(True)
        out2 = matmul(measurements[idx], out1)
        out1 = inner_prod(out1_copy, out2) # this is already <Psi|0><0|Psi>
        #out1_star = conj(out1)
        out2 = 1-torch.abs(real(out1))
        loss = loss + out2
    return loss/len(x_train)

class Model():
    def __init__(self, params, width=2, layers=2, device="cpu"):
        if width%2 != 0:
            raise ValueError("So far only implemented for even qubit number!!!")
        self.width = width
        self.layers = layers
        self.measures = Measurements(width, device=device)
        # For each layer we itterate through all the qubits and apply Rotations, which depend on two times
        # 3 parameters, because Rot(\theta + W*x)
        self.CZ_list = [self.CZ_layer(0, device=device), self.CZ_layer(1, device=device)]

    def Rot_layer(self, params, layer, x=None, device="cpu"):
        Rot_Matrix = None
        for par in params[layer]:
            angles = par[0] + par[1]*x.float()
            if Rot_Matrix is not None:
                Rot_Matrix = kronecker_prod( R3(angles), Rot_Matrix)
            else:
                Rot_Matrix = R3(angles)
        return Rot_Matrix

    def CZ_layer(self, layer, device="cpu"):
        CZ_Matrix = 1.0
        for i in range(int(self.width/2)):
            if layer%2==0:
                CZ_Matrix *= CZ(self.width, c=2*i, t=2*i+1)
            if layer%2 == 1:
                CZ_Matrix *= CZ(self.width, c=2*i+1, t=(2*i+2)%self.width)
        return make_complex(CZ_Matrix).to(device)

    def build_circuit(self, params, x=None, device='cpu'):
        circuit = matmul(self.CZ_layer(0, device=device), self.Rot_layer(params, 0, x=x, device=device))
        for i in range(1,self.layers):
            circuit = matmul(self.Rot_layer(params, i, x=x, device=device), circuit)
            circuit = matmul(self.CZ_list[i%2], circuit)
        return circuit.to(device)

class Model_Z_measure():
    def __init__(self, params, width=2, layers=2, device="cpu"):
        if width%2 != 0:
            raise ValueError("So far only implemented for even qubit number!!!")
        self.width = width
        self.layers = layers
        self.measures = Measurement_Z(width, device=device)
        # For each layer we itterate through all the qubits and apply Rotations, which depend on two times
        # 3 parameters, because Rot(\theta + W*x)
        self.CZ_list = [self.CZ_layer(0, device=device), self.CZ_layer(1, device=device)]

    def Rot_layer(self, params, layer, x=None, device="cpu"):
        Rot_Matrix = None
        for par in params[layer]:
            angles = par[0] + par[1]*x.float()
            if Rot_Matrix is not None:
                Rot_Matrix = kronecker_prod( R3(angles), Rot_Matrix)
            else:
                Rot_Matrix = R3(angles)
        return Rot_Matrix

    def CZ_layer(self, layer, device="cpu"):
        CZ_Matrix = 1.0
        for i in range(int(self.width/2)):
            if layer%2==0:
                CZ_Matrix *= CZ(self.width, c=2*i, t=2*i+1)
            if layer%2 == 1:
                CZ_Matrix *= CZ(self.width, c=2*i+1, t=(2*i+2)%self.width)
        return make_complex(CZ_Matrix).to(device)

    def build_circuit(self, params, x=None, device='cpu'):
        circuit = matmul(self.CZ_layer(0, device=device), self.Rot_layer(params, 0, x=x, device=device))
        for i in range(1,self.layers):
            circuit = matmul(self.Rot_layer(params, i, x=x, device=device), circuit)
            circuit = matmul(self.CZ_list[i%2], circuit)
        return circuit.to(device)